<a href="https://colab.research.google.com/github/KamilShape/building_style_recognition/blob/main/computer_vision_my_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import requests
from bs4 import BeautifulSoup
from PIL import Image
from io import BytesIO
import random
import os
import numpy as np


### Connecting to google drive

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Getting Images

In [7]:
styles = ['greek-architecture', 'gothic-architecture', 'baroque-architecture', 'victorian-architecture', 'modern architecture']

In [8]:
def parse_url(content):
  soup = BeautifulSoup(content, 'html.parser')
  imgs = soup.find_all('img', {'class': 'grid-image'})
  return [img['src'] for img in imgs]

In [10]:
def get_page(style, page):
  for page in range(1, page + 1):
    url = f'https://www.freeimages.com/search/{style}/{page}'
    try:
      response = requests.get(url)
      content = response.content
      return content
    except:
      print('Error! Wrong url.')

In [11]:
def get_image(photo_url, path, file_name, style):
  image_response = requests.get(photo_url)
  image_content = image_response.content
  img = Image.open(BytesIO(image_content))
  # print(img.mode)
  if img.mode == 'RGB':
    os.makedirs(f'{path}/{style}', exist_ok=True)
    img.save(f'{path}/{style}/{file_name}', 'png')

In [12]:
train_path = 'drive/MyDrive/architecture_photos/data/train'

In [13]:
def get_photos(style, page, path):
  content = get_page(style, page)
  images = parse_url(content)
  for image in images:
    if len(image) > 5:
      file_name = image.split('/')[-1]
      get_image(image, train_path, file_name, style)


In [100]:
#DOWNLOADING ALL PHOTOS
for style in styles:
  get_photos(style, 10, train_path)

### Test and train data sets

In [3]:
test_path = 'drive/MyDrive/architecture_photos/data/test'

In [ ]:
os.makedirs(f'{test_path}')

In [5]:
def create_test_set(styles, train_path, test_path, set_size):
  for style in styles:
    files_folder = os.listdir(f'{train_path}/{style}')
    for n in range(0, round(len(files_folder)*set_size)+1):
      files_folder = os.listdir(f'{train_path}/{style}')
      photo_number = np.random.randint(len(files_folder))
      os.makedirs(f'{test_path}/{style}', exist_ok=True)
      os.replace(f'{train_path}/{style}/{files_folder[photo_number]}', f'{test_path}/{style}/{files_folder[photo_number]}')

In [105]:
create_test_set(styles, train_path, test_path, 0.15)

Model 

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow import keras

In [15]:
img_height = 128
img_width = 128
batch_size = 8

In [16]:
train_ds = keras.utils.image_dataset_from_directory(
    train_path,
    validation_split = 0.1,
    seed=0,
    subset='training',
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

val_ds = keras.utils.image_dataset_from_directory(
    train_path,
    validation_split = 0.1,
    seed=0,
    subset='validation',
    image_size = (img_height, img_width),
    batch_size = batch_size,
    label_mode = 'categorical')

Found 214 files belonging to 5 classes.
Using 193 files for training.
Found 214 files belonging to 5 classes.
Using 21 files for validation.


In [22]:
model = Sequential()
model.add(layers.Rescaling(1./255))
model.add(layers.Conv2D(32, (3,3), input_shape = (img_width, img_height, 3), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Conv2D(16, (3,3), padding='same', activation='relu'))
model.add(layers.MaxPool2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))

In [23]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

In [24]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 10,
    batch_size = batch_size
)

Epoch 1/10
25/25 [==============================] - 39s 752ms/step - loss: 1.6606 - accuracy: 0.1762 - val_loss: 1.6090 - val_accuracy: 0.1429
Epoch 2/10
25/25 [==============================] - 0s 15ms/step - loss: 1.5840 - accuracy: 0.3264 - val_loss: 1.5205 - val_accuracy: 0.3333
Epoch 3/10
25/25 [==============================] - 0s 16ms/step - loss: 1.3811 - accuracy: 0.5233 - val_loss: 1.6644 - val_accuracy: 0.1905
Epoch 4/10
25/25 [==============================] - 0s 15ms/step - loss: 0.8406 - accuracy: 0.7306 - val_loss: 1.5997 - val_accuracy: 0.3333
Epoch 5/10
25/25 [==============================] - 0s 15ms/step - loss: 0.4325 - accuracy: 0.8860 - val_loss: 1.4691 - val_accuracy: 0.3810
Epoch 6/10
25/25 [==============================] - 0s 15ms/step - loss: 0.2668 - accuracy: 0.9119 - val_loss: 2.0257 - val_accuracy: 0.2857
Epoch 7/10
25/25 [==============================] - 0s 15ms/step - loss: 0.1375 - accuracy: 0.9689 - val_loss: 2.9946 - val_accuracy: 0.3333
Epoch 8/10
